<a href="https://colab.research.google.com/github/rishubhkhurana/EVA/blob/master/S4/EVA4_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self,p=0.1):
        super(Net, self).__init__()
        #  first conv block extracting edges/gradients 
        lyrs1=[]
        lyrs1.append(nn.Conv2d(1, 16, 3)) #28,26,3 
        lyrs1.append(nn.BatchNorm2d(16))
        lyrs1.append(nn.ReLU())
        lyrs1.append(nn.Dropout(p=p))
        lyrs1.append(nn.Conv2d(16, 32, 3)) #26,24,5
        lyrs1.append(nn.BatchNorm2d(32))
        lyrs1.append(nn.ReLU())
        lyrs1.append(nn.Dropout(p=p))
        self.block1 = nn.Sequential(*lyrs1)

        # first transition block mixing channels
        lyrs2=[] 
        lyrs2.append(nn.MaxPool2d(2, 2)) #24,12,6
        lyrs2.append(nn.Conv2d(32,16,1)) #12,12,6
        lyrs2.append(nn.BatchNorm2d(16))
        lyrs2.append(nn.ReLU())
        lyrs2.append(nn.Dropout(p=p))
        self.tblock1 = nn.Sequential(*lyrs2)

        #second conv block extracting textures
        lyrs3=[]
        lyrs3.append(nn.Conv2d(16, 16, 3))#12,10,10
        lyrs3.append(nn.BatchNorm2d(16))
        lyrs3.append(nn.ReLU())
        lyrs3.append(nn.Dropout(p=p))
        lyrs3.append(nn.Conv2d(16,16,3))#10,8,14
        lyrs3.append(nn.BatchNorm2d(16))
        lyrs3.append(nn.ReLU())
        lyrs3.append(nn.Dropout(p=p))
        lyrs3.append(nn.Conv2d(16,16,3))#8,6,18
        lyrs3.append(nn.BatchNorm2d(16))
        lyrs3.append(nn.ReLU())
        lyrs3.append(nn.Dropout(p=p))
        #lyrs3.append(nn.Conv2d(16,16,3))#6,4,22
        #lyrs3.append(nn.BatchNorm2d(16))
        #lyrs3.append(nn.ReLU())
        #lyrs3.append(nn.Dropout(p=p))
        lyrs3.append(nn.Conv2d(16,10,6))#6,1,28
        self.block2 = nn.Sequential(*lyrs3)

        
       

    def forward(self, x):
        x = self.tblock1(self.block1(x))
        x = self.block2(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [16]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
       BatchNorm2d-2           [-1, 16, 26, 26]              32
              ReLU-3           [-1, 16, 26, 26]               0
           Dropout-4           [-1, 16, 26, 26]               0
            Conv2d-5           [-1, 32, 24, 24]           4,640
       BatchNorm2d-6           [-1, 32, 24, 24]              64
              ReLU-7           [-1, 32, 24, 24]               0
           Dropout-8           [-1, 32, 24, 24]               0
         MaxPool2d-9           [-1, 32, 12, 12]               0
           Conv2d-10           [-1, 16, 12, 12]             528
      BatchNorm2d-11           [-1, 16, 12, 12]              32
             ReLU-12           [-1, 16, 12, 12]               0
          Dropout-13           [-1, 16, 12, 12]               0
           Conv2d-14           [-1, 16,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:53: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader,name='Test'):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\n{} set: Average loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(name,
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [18]:

model = Net(p=0.1).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.9)
#range_bar = tqdm(range(1,21))
for epoch in range(1,21):
    train(model, device, train_loader, optimizer, epoch)
    #test(model,device,train_loader,name='Train')
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0675, Accuracy: 9812/10000 (98.120%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0448, Accuracy: 9859/10000 (98.590%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0188, Accuracy: 9936/10000 (99.360%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0170, Accuracy: 9944/10000 (99.440%)



loss=0.0990472063422203 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 40.53it/s]



Test set: Average loss: 0.0168, Accuracy: 9942/10000 (99.420%)

